In [ ]:
import glob
import os
from collections import defaultdict
from tqdm import tqdm

from pymatreader import read_mat
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pickle import load

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import timm

from model import OneEncoderNet, TwoEncoderNet, TemporalNet

data_dir = '../../data/test'
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

CROP_LEN_ = 250

label_dict = {
    0: 'frontside_kickturn',
    1: 'backside_kickturn',
    2: 'pumping'
}

c:\Users\teraoka_r\workspace\development\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def get_data(data_dir):
    data_dict = {}

    for mat_data in glob.glob(f'{data_dir}/*.mat'):
        subject = mat_data.split('\\')[-1].split('.')[0]
        data = read_mat(mat_data)
        data_dict[subject] = data['data']

    ch_names = [c.replace(' ', '') for c in data['ch_labels']]
    diff_list = [
        #横方向
        'F3_F4',
        'FCz_FC1', 'FCz_FC2', 'FCz_FC3', 'FCz_FC4', 'FCz_FC5', 'FCz_FC6', 'FC1_FC2', 'FC3_FC4', 'FC5_FC6',
        'Cz_C1', 'Cz_C2', 'Cz_C3', 'Cz_C4', 'Cz_C5', 'Cz_C6', 'C1_C2', 'C3_C4', 'C5_C6',
        'CPz_CP1', 'CPz_CP2', 'CPz_CP3', 'CPz_CP4', 'CPz_CP5', 'CPz_CP6', 'CP1_CP2', 'CP3_CP4', 'CP5_CP6',
        'P3_P4',
        #縦方向
        'Cz_FCz', 'C1_FC1', 'C2_FC2', 'C3_FC3', 'C4_FC4', 'C5_FC5', 'C6_FC6',
        'Cz_CPz', 'C1_CP1', 'C2_CP2', 'C3_CP3', 'C4_CP4', 'C5_CP5', 'C6_CP6',
        'FCz_CPz', 'FC1_CP1', 'FC2_CP2', 'FC3_CP3', 'FC4_CP4', 'FC5_CP5', 'FC6_CP6',
    ]

    use_ch = []
    for item in diff_list:
        ch1 = item.split('_')[0]
        ch2 = item.split('_')[1]
        use_ch.append(ch1)
        use_ch.append(ch2)

    use_ch = list(set(use_ch))
    use_ch_dict = {ch_names[idx]:idx for idx in range(len(ch_names)) if ch_names[idx] in use_ch}
    
    return data_dict, diff_list, use_ch_dict


class SkateDataset(Dataset):
    def __init__(self, data, diff_list, use_ch_dict):
        self.diff_list = diff_list
        self.use_ch_dict = use_ch_dict

        for i, data_ in enumerate(data):
            median = np.median(data_, axis=1).reshape(72, 1)
            q1 = np.percentile(data_, 25, axis=1)
            q3 = np.percentile(data_, 75, axis=1)
            iqr = (q3 - q1).reshape(72, 1)
            iqr = np.where(iqr==0, 1, iqr)
            data_ = (data_ - median) / iqr 
            data[i] = data_

        self.data = data

    def __len__(self):
        return self.data.shape[0]

    def __getitem__(self, index):
        data = self.data[index]
        
        return data
    

In [3]:
data_dict, diff_list, use_ch_dict = get_data(data_dir)

In [4]:
def get_nn_model(fold):
    model_list = []
    model = OneEncoderNet(in_channels=72)
    load_weights = torch.load(f'../1st_train_1D/one_encoder/model/model{fold}.pth', map_location=device)
    model.load_state_dict(load_weights)
    model = model.eval().to(device)
    model_list.append(model)

    model = TwoEncoderNet(in_channels=72)
    load_weights = torch.load(f'../1st_train_1D/two_encoder/model/model{fold}.pth', map_location=device)
    model.load_state_dict(load_weights)
    model = model.eval().to(device)
    model_list.append(model)

    model = TemporalNet(model_name='tf_efficientnet_b0.in1k', in_channels=72)
    load_weights = torch.load(f'../1st_train_1Dtemporal/tf_efficientnet_b0/model/model{fold}.pth', map_location=device)
    model.load_state_dict(load_weights)
    model = model.eval().to(device)
    model_list.append(model)

    model = TemporalNet(model_name='tf_efficientnet_b3.in1k', in_channels=72)
    load_weights = torch.load(f'../1st_train_1Dtemporal/tf_efficientnet_b3/model/model{fold}.pth', map_location=device)
    model.load_state_dict(load_weights)
    model = model.eval().to(device)
    model_list.append(model)

    model = TemporalNet(model_name='tf_efficientnetv2_b0.in1k', in_channels=72)
    load_weights = torch.load(f'../1st_train_1Dtemporal/tf_efficientnetv2_b0/model/model{fold}.pth', map_location=device)
    model.load_state_dict(load_weights)
    model = model.eval().to(device)
    model_list.append(model)

    model = TemporalNet(model_name='tf_efficientnetv2_b3.in1k', in_channels=72)
    load_weights = torch.load(f'../1st_train_1Dtemporal/tf_efficientnetv2_b3/model/model{fold}.pth', map_location=device)
    model.load_state_dict(load_weights)
    model = model.eval().to(device)
    model_list.append(model)

    model = TemporalNet(model_name='efficientvit_b1.r256_in1k', in_channels=72)
    load_weights = torch.load(f'../1st_train_1Dtemporal/efficientvit_b1/model/model{fold}.pth', map_location=device)
    model.load_state_dict(load_weights)
    model = model.eval().to(device)
    model_list.append(model)

    model = TemporalNet(model_name='efficientvit_b3.r256_in1k', in_channels=72)
    load_weights = torch.load(f'../1st_train_1Dtemporal/efficientvit_b3/model/model{fold}.pth', map_location=device)
    model.load_state_dict(load_weights)
    model = model.eval().to(device)
    model_list.append(model)

    return model_list


def get_lgb_model(fold):
    model_list = []
    temp = []
    for fold_ in range(3):
        model = load(open(f'../1st_train_1D/one_encoder/lgb_model/lgb_fold{fold}_{fold_}', 'rb'))
        temp.append(model)
    model_list.append(temp)

    temp = []
    for fold_ in range(3):
        model = load(open(f'../1st_train_1D/two_encoder/lgb_model/lgb_fold{fold}_{fold_}', 'rb'))
        temp.append(model)
    model_list.append(temp)

    temp = []
    for fold_ in range(3):
        model = load(open(f'../1st_train_1Dtemporal/tf_efficientnet_b0/lgb_model/lgb_fold{fold}_{fold_}', 'rb'))
        temp.append(model)
    model_list.append(temp)

    temp = []
    for fold_ in range(3):
        model = load(open(f'../1st_train_1Dtemporal/tf_efficientnet_b3/lgb_model/lgb_fold{fold}_{fold_}', 'rb'))
        temp.append(model)
    model_list.append(temp)

    temp = []
    for fold_ in range(3):
        model = load(open(f'../1st_train_1Dtemporal/tf_efficientnetv2_b0/lgb_model/lgb_fold{fold}_{fold_}', 'rb'))
        temp.append(model)
    model_list.append(temp)

    temp = []
    for fold_ in range(3):
        model = load(open(f'../1st_train_1Dtemporal/tf_efficientnetv2_b3/lgb_model/lgb_fold{fold}_{fold_}', 'rb'))
        temp.append(model)
    model_list.append(temp)

    temp = []
    for fold_ in range(3):
        model = load(open(f'../1st_train_1Dtemporal/efficientvit_b1/lgb_model/lgb_fold{fold}_{fold_}', 'rb'))
        temp.append(model)
    model_list.append(temp)

    temp = []
    for fold_ in range(3):
        model = load(open(f'../1st_train_1Dtemporal/efficientvit_b3/lgb_model/lgb_fold{fold}_{fold_}', 'rb'))
        temp.append(model)
    model_list.append(temp)

    return model_list


def get_weight():
    weight_list = []
    weight = np.load(f'../1st_train_1D/one_encoder/output/weight.npy', allow_pickle=True)
    weight_list.append(weight)

    weight = np.load(f'../1st_train_1D/two_encoder/output/weight.npy', allow_pickle=True)
    weight_list.append(weight)

    weight = np.load(f'../1st_train_1Dtemporal/tf_efficientnet_b0/output/weight.npy', allow_pickle=True)
    weight_list.append(weight)

    weight = np.load(f'../1st_train_1Dtemporal/tf_efficientnet_b3/output/weight.npy', allow_pickle=True)
    weight_list.append(weight)

    weight = np.load(f'../1st_train_1Dtemporal/tf_efficientnetv2_b0/output/weight.npy', allow_pickle=True)
    weight_list.append(weight)

    weight = np.load(f'../1st_train_1Dtemporal/tf_efficientnetv2_b3/output/weight.npy', allow_pickle=True)
    weight_list.append(weight)

    weight = np.load(f'../1st_train_1Dtemporal/efficientvit_b1/output/weight.npy', allow_pickle=True)
    weight_list.append(weight)

    weight = np.load(f'../1st_train_1Dtemporal/efficientvit_b3/output/weight.npy', allow_pickle=True)
    weight_list.append(weight)

    return weight_list


def predict(X, nn_model_list, lgb_model_list, weight_list, fold):
    preds = []
    for nn_model, lgb_model, weight in zip(nn_model_list, lgb_model_list, weight_list):
        with torch.no_grad():
            nn_pred, features = nn_model(X)
        nn_pred = nn.functional.softmax(nn_pred, dim=1).detach().cpu().numpy()
        
        lgb_pred = []
        for model in lgb_model:
            proba = model.predict_proba(features.detach().cpu().numpy(), num_iteration=model.best_iteration_)
            lgb_pred.append(proba)
        lgb_pred = np.mean(np.stack(lgb_pred, axis=0), axis=0)

        y_pred = weight[fold] * nn_pred + (1 - weight[fold]) * lgb_pred

        preds.append(y_pred)
    preds = torch.from_numpy(np.stack(preds, axis=0))

    return preds

In [ ]:
target_list = ['subject0', 'subject1', 'subject2', 'subject3', 'subject4']
weight_list = get_weight()
predict_list = []
for target in target_list:
    print(f'calculating -> {target}')
    data = data_dict[target]
    preds = []
    for fold in range(3):
        dataset = SkateDataset(data, diff_list, use_ch_dict)
        X = dataset.data
        X = torch.from_numpy(X).to(device).float()

        nn_model_list = get_nn_model(fold)
        lgb_model_list = get_lgb_model(fold)
        y_pred = predict(X, nn_model_list, lgb_model_list, weight_list, fold)
        preds.append(y_pred)
    preds = torch.argmax(torch.mean(torch.cat(preds, dim=0), dim=0), dim=1)
    for i, pred in enumerate(preds):
        if i < 10:
            number = f'00{i}'
        elif i < 100:
            number = f'0{i}'
        else:
            number = str(i)
        target_name = f'{target}_{number}'
        label = label_dict[pred.item()]
        predict_list.append([target_name, label])

In [6]:
df = pd.DataFrame(predict_list)
df.to_csv('submit.csv', header=None, index=None)